# Build and save the mean homography matrix

In [ ]:
import numpy as np
h = np.eye(3, dtype=float)

h[2, 2] = 1
h[2, 1] = -1e-6
h[2, 0] = 1e-6

h[1, 2] = -0.78
h[1, 1] = 0.999
h[1, 0] = -0.015

h[0, 2] = -5.2
h[0, 1] = 0.017
h[0, 0] = 0.9998

h

# Save warped results (with mean homography) as .npz file

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import torch
from tqdm import tqdm
from kornia.geometry import warp_perspective


path_to_files = Path('rawData') / 'B'
src = np.load(path_to_files / 'src.npz')
dest = np.load(path_to_files / 'dest.npz')
print(f"src {list(src.keys())}, length {len(src['frames'])}")
print(f"dest {list(dest.keys())}, length {len(dest['frames'])}")


In [ ]:
# Warp all src to dest using the mean h
path_to_save_warped = path_to_files / 'warped'
path_to_save_warped.mkdir(exist_ok=True)
(path_to_save_warped / 'pan').mkdir(exist_ok=True)
(path_to_save_warped / 'orig').mkdir(exist_ok=True)
(path_to_save_warped / 'mono').mkdir(exist_ok=True)
indices = np.arange(len(src['frames']))
for idx in tqdm(indices):
    ret_val = cv2.warpPerspective(src['frames'][idx], np.linalg.inv(h), list(reversed(src['frames'][idx].shape[-2:])))
    mask = cv2.warpPerspective(
        np.ones_like(src['frames'][idx]),
        np.linalg.inv(h), list(reversed(src['frames'][idx].shape[-2:])),
        flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=0).astype(bool)
    np.savez(path_to_save_warped / 'pan' / f'{idx}.npz', **{'image': ret_val, 'fpa': src['fpa'][idx], 'mask':mask})
    np.savez(path_to_save_warped / 'mono' / f'{idx}.npz', **{'image': dest['frames'][idx], 'fpa': dest['fpa'][idx]})
    np.save(path_to_save_warped / 'orig' / f'{idx}.npy', src['frames'][idx])


Sanity check for saving

In [ ]:
indices = np.arange(len(src['frames']))
# np.random.shuffle(indices)
n_subplot = 5
for idx in indices[:3]:
    src_ = np.load(path_to_save_warped / 'pan' / f'{idx}.npz')['image'].astype(float)
    orig_ = np.load(path_to_save_warped / 'orig' / f'{idx}.npy')
    mask_ = np.load(path_to_save_warped / 'pan' / f'{idx}.npz')['mask'].astype(bool)
    dest_ = np.load(path_to_save_warped / 'mono' / f'{idx}.npz')['image'].astype(float)
    plt.subplots(1, n_subplot, figsize=(n_subplot*5, 5))
    plt.subplot(1, n_subplot, 1)
    plt.imshow(orig_)
    plt.title(f'Orig {np.ptp(orig_)=}')
    plt.subplot(1, n_subplot, 2)
    plt.imshow(src_, vmin=src_[mask].min(), vmax=src_[mask].max())
    plt.title(f'Warped src {np.ptp(src_[mask])=}')
    plt.subplot(1, n_subplot, 3)
    plt.imshow(dest_)
    plt.title(f'dest {np.ptp(dest_)=}')
    # plt.subplot(1, 3, 3)
    # diff = np.abs(src_ - dest_)
    # plt.imshow(diff, vmin=diff[mask_].min(), vmax=diff[mask_].max())
    # plt.title(f'diff {np.ptp(diff[mask_])=}')

    plt.subplot(1, n_subplot, 4)
    # normalize the src and dest and than diff
    src__ = (src_ - src_[mask].min()) / (src_[mask].max() - src_[mask].min())
    dest__ = (dest_ - dest_.min()) / (dest_.max() - dest_.min())
    diff = np.abs(src__ - dest__)
    plt.imshow(diff, vmin=diff[mask_].min(), vmax=diff[mask_].max())
    plt.title(f'Normalized diff {np.ptp(diff[mask_])=:.2g}')
    
    plt.subplot(1, n_subplot, 5)
    # normalize the src and dest and than diff
    orig__ = (orig_ - orig_[mask].min()) / (orig_[mask].max() - orig_[mask].min())
    dest__ = (dest_ - dest_.min()) / (dest_.max() - dest_.min())
    diff = np.abs(orig__ - dest__)
    plt.imshow(diff, vmin=diff[mask_].min(), vmax=diff[mask_].max())
    plt.title(f'Norm diff orig-dest {np.ptp(diff[mask_])=:.2g}')
    
    
    plt.show()
    plt.close()